In [1]:
import pickle
from neo4j import GraphDatabase
from dotenv import dotenv_values
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cdlib import algorithms, viz, evaluation

from hp_nlp_graph.neo4j import add_metrics_to_neo4j

Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'wurlitzer', 'graph_tool', 'bayanpy'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'infomap', 'wurlitzer'}


In [3]:
book_number = 3
interactions_by_chapter = pickle.load(
    open(f"data/processed/{book_number}/interactions_by_chapter.pkl", "rb")
)

In [4]:
neo4j_config = dotenv_values(".env")
driver = GraphDatabase.driver(
    neo4j_config["NEO4J_URL"],
    auth=(neo4j_config["NEO4J_USER"], neo4j_config["NEO4J_PASSWORD"]),
)

In [5]:
dfs = []
for chapter, distances in interactions_by_chapter.items():
    df = pd.DataFrame.from_dict(
        dict(distances), orient="index", columns=["weight"]
    ).reset_index(names=["characters"])
    df[["source", "target"]] = pd.DataFrame(df.characters.tolist(), index=df.index)
    df["chapter"] = chapter
    df["book"] = 1
    df = df[["source", "target", "weight", "chapter", "book"]]
    dfs.append(df)
df = (
    pd.concat(dfs)
    .groupby(["source", "target"])
    .weight.sum()
    .sort_values(ascending=False)
    .reset_index()
)

In [6]:
G = nx.from_pandas_edgelist(df, "source", "target", ["weight"], create_using=nx.Graph())

In [7]:
eigen_centrality = nx.eigenvector_centrality(G, weight="weight")
betweenness_centrality = nx.betweenness_centrality(G, weight="weight")
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
pagerank = nx.pagerank(G, weight="weight")
hub_centrality, authority_centrality = nx.hits(G)
degree = dict(nx.degree(G))
weighted_degree = dict(nx.degree(G, weight="weight"))

metrics = {
    "eigen_centrality": eigen_centrality,
    "betweenness_centrality": betweenness_centrality,
    "degree_centrality": degree_centrality,
    "closeness_centrality": closeness_centrality,
    "pagerank": pagerank,
    "hub": hub_centrality,
    "authority": authority_centrality,
    "degree": degree,
    "weighted_degree": weighted_degree,
}
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df.index.name = "name"
metrics_df

,eigen_centrality,betweenness_centrality,degree_centrality,closeness_centrality,pagerank,hub,authority,degree,weighted_degree
name,,,,,,,,,
Harry Potter,0.595373,0.148828,0.865169,0.881188,0.203824,0.190030,0.190030,77,5932
Ronald Weasley,0.516591,0.089374,0.617978,0.723577,0.105627,0.164882,0.164882,55,3340
Hermione Granger,0.511393,0.071834,0.561798,0.695312,0.099515,0.163223,0.163223,50,3149
Remus Lupin,0.158884,0.067854,0.393258,0.613793,0.038898,0.050711,0.050711,35,1199
Sirius Black,0.159712,0.024783,0.426966,0.626761,0.041523,0.050976,0.050976,38,1273
...,...,...,...,...,...,...,...,...,...
Cuthbert Binns,0.000514,0.016785,0.033708,0.483696,0.001750,0.000164,0.000164,3,3
Charles Weasley,0.000481,0.027190,0.044944,0.491713,0.001808,0.000154,0.000154,4,4
Bathilda Bagshot,0.000241,0.000000,0.011236,0.470899,0.001696,0.000077,0.000077,1,1


In [8]:
louvain_comms = algorithms.louvain(G, weight="weight")
louvain = pd.DataFrame.from_dict(
    dict(louvain_comms.to_node_community_map()),
    orient="index",
    columns=["louvain"],
)
louvain.index.name = "name"

In [9]:
leiden_comms = algorithms.leiden(G, weights="weight")
leiden = pd.DataFrame.from_dict(
    dict(leiden_comms.to_node_community_map()),
    orient="index",
    columns=["leiden"],
)
leiden.index.name = "name"

In [10]:
girvan_newman_comms = algorithms.girvan_newman(G, level=5)
girvan_newman = pd.DataFrame.from_dict(
    dict(girvan_newman_comms.to_node_community_map()),
    orient="index",
    columns=["girvan_newman"],
)
girvan_newman.index.name = "name"

In [11]:
spectral_comms = algorithms.spectral(G, kmax=8)
spectral = pd.DataFrame.from_dict(
    dict(spectral_comms.to_node_community_map()),
    orient="index",
    columns=["spectral"],
)
spectral.index.name = "name"

In [12]:
metrics_df = metrics_df.join([louvain, leiden, girvan_newman, spectral])

In [13]:
metrics_df.to_csv(f"data/processed/{book_number}/metrics.csv")

In [13]:
add_metrics_to_neo4j(driver, metrics_df.reset_index().to_dict("records"))

In [14]:
for col in metrics_df.columns:
    nx.set_node_attributes(G, metrics_df[col].to_dict(), col)

In [15]:
nx.write_gexf(G, f"data/processed/{book_number}/graph.gexf")